In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import networkx as nx
import rtsvg
import os
import json
rt = rtsvg.RACETrack()

In [ ]:
# Parse all of the coverage files
coverage_files  = os.listdir('../coverage/')
coverage_lu     = {}
code_files_seen = set()
for _file_ in coverage_files:
    _json_  = json.loads(open('../coverage/' + _file_).read())
    coverage_lu[_file_] = _json_
    code_files_seen = code_files_seen.union(_json_['files'].keys())

# Remove any tests files
to_remove = set()
for _file_ in code_files_seen:
    if _file_.startswith('tests/'): to_remove.add(_file_)
code_files_seen = code_files_seen - to_remove

# Convert them into a dataframe that determines the timestamp, code_file, and percent_covered
_lu_ = {'timestamp':[], 'code_file':[], 'percent_covered':[]}
for _file_ in coverage_files:
    _json_      = coverage_lu[_file_]
    _timestamp_ = _json_['meta']['timestamp']
    for _code_file_ in code_files_seen:
        if _code_file_ in _json_['files']:
            _percent_covered_ = _json_['files'][_code_file_]['summary']['percent_covered']
            _lu_['timestamp'].append(_timestamp_), _lu_['code_file'].append(_code_file_), _lu_['percent_covered'].append(_percent_covered_)
df = pl.DataFrame(_lu_)
df = rt.columnsAreTimestamps(df, ['timestamp'])
last_timestamp_seen = df['timestamp'].max()

# Plot them out
rt.tile([rt.xy(df, x_field='timestamp', y_field='percent_covered', line_groupby_field='code_file', line_groupby_w=4, color_by='code_file', dot_size=None, w=1024, h=512),
         rt.histogram(df.filter(pl.col('timestamp') == last_timestamp_seen), bin_by='code_file', count_by='percent_covered', color_by='code_file', w=512, h=512)])
